In [1]:
import tensorflow as tf

In [2]:
x_queue = tf.train.string_input_producer(["./train.csv"])
reader1 = tf.TextLineReader(skip_header_lines=1)

keyx, valuex = reader1.read(x_queue)
x = tf.decode_csv(
    valuex, record_defaults=[[0.] for i in range(653)])



In [3]:
batch_size = 100
min_after_dequeue = 1000
capacity = min_after_dequeue + 3 * batch_size
train= tf.train.batch([x],batch_size=100, capacity=capacity)

In [4]:
SEED = 66478

weights = tf.Variable( tf.random_uniform([651,1], -1.0, 1.0))
biases = tf.Variable(tf.zeros([1], dtype="float32"))
feature = train[:,1:-1]
z = tf.add(tf.matmul(feature,weights),biases)
y_hat = tf.squeeze(tf.div(1.0, 1.0 + tf.exp(-z)))
y = train[:,-1]
loss = -tf.reduce_mean((y*tf.log(y_hat) + (1-y)*tf.log(1-y_hat)))
print(loss)
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

Tensor("Neg_1:0", shape=(), dtype=float32)


In [5]:
def get_data ():
    with tf.Session() as sess:  
        #sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()  #创建一个协调器，管理线程 
        threads = tf.train.start_queue_runners(coord=coord)
        return sess.run([feature,y])


In [9]:
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    #coord = tf.train.Coordinator()  #创建一个协调器，管理线程 
    #threads = tf.train.start_queue_runners(coord=coord)  #启动QueueRunner, 此时文件名队列已经进队。 
    #saver = tf.train.Saver()
    #ckpt = tf.train.get_checkpoint_state('./check/')
    """
    if ckpt:
        saver.restore(sess, tf.train.latest_checkpoint('./check/'))
        print('loaded')
    else:
        tf.global_variables_initializer().run()"""
with tf.Session() as sess:
    saver = tf.train.Saver()
    ckpt = tf.train.get_checkpoint_state('./check/')
    if ckpt:
        saver.restore(sess, tf.train.latest_checkpoint('./check/'))
        print('loaded')
    else:
        tf.global_variables_initializer().run()
    #sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()  #创建一个协调器，管理线程 
    threads = tf.train.start_queue_runners(coord=coord)  #启动QueueRunner, 此时文件名队列已经进队。 
    for i in range(100000):
        if (i % 40 == 0):
            print (i, "th iteration done")
            e_val = sess.run(loss)
            print(e_val)
            #print(sess.run(weights))
            #a,b,c = sess.run([feature,y,y_hat])
            #print(a,b,c)
        sess.run(train_step)
        
    
        if (i + 1) % 1000 == 0:
            saver.save(sess, './check/' + 'model'+ str(i) +'.ckpt',
                           global_step=i+1)
        
            
            
            print(1)
    coord.request_stop()  
    coord.join(threads)

INFO:tensorflow:Restoring parameters from ./check/model9999.ckpt-10000
loaded
0 th iteration done
0.785378
40 th iteration done
0.656405
80 th iteration done
0.810528
120 th iteration done
0.687638
160 th iteration done
0.691123
200 th iteration done
0.647959
240 th iteration done
0.722363
280 th iteration done
0.681473
320 th iteration done
0.58069
360 th iteration done
0.662922
400 th iteration done
0.94314
440 th iteration done
0.799493
480 th iteration done
0.547177
520 th iteration done
0.695431
560 th iteration done
0.855313
600 th iteration done
0.821785
640 th iteration done
0.709668
680 th iteration done
0.762043
720 th iteration done
0.72424
760 th iteration done
0.653213
800 th iteration done
0.554303
840 th iteration done
0.687516
880 th iteration done
0.506522
920 th iteration done
0.551773
960 th iteration done
0.59029
1
1000 th iteration done
0.550808
1040 th iteration done
0.568632
1080 th iteration done
0.57936
1120 th iteration done
0.640143
1160 th iteration done
0.5

KeyboardInterrupt: 

In [ ]:
def predict():
    x_queue = tf.train.string_input_producer(["./valid.csv"])
    reader2 = tf.TextLineReader(skip_header_lines=1)
    keyx, valuex = reader1.read(x_queue)
    x = tf.decode_csv(
        valuex, record_defaults=[[0.] for i in range(653)])